# Train relevance classifier

This script trains the relevance classifier meant to identify DOI plans. 

In [1]:
import pandas as pd
import os
import classify

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


## Label documents

Do not overwrite labelled document set unless you want to relabel. relabel_docs = False protects labelled dataset. Sample size is currently 140. You may decide to add additional DOI plans. This can be done with add_extra_trues. 

In [2]:
relabel_docs = False
sample_size = 140
add_extra_trues = True
num_new_true = 120

In [3]:
if relabel_docs:
    text_scraped = pd.read_csv(os.path.join(os.getcwd(), '..', '..', 'data', 'text_scraped.csv'))
    docs_to_label = text_scraped.sample(n = sample_size)
    docs_to_label.to_csv(os.path.join(os.getcwd(), '..', '..', 'data', 'docs_to_label.csv'))
    labeled_docs = docs_to_label

In [4]:
labeled_docs = pd.read_csv(os.path.join(os.getcwd(), '..', '..', 'data', 'docs_to_label.csv'))
labeled_docs['innovation_plan'] = [
         False, False, False, True, False,
         False, True, False, False, False,
         False, False, False, False, False,
         False, False, False, False, False,
    
         True, True, False, True, False,
         False, True, True, True, False,
         False, False, False, True, False,
         False, False, False, False, False,
    
         False, False, False, False, False,
         False, False, False, False, False,
         False, False, False, False, False,
         False, False, False, True, False,
    
         False, True, False, False, True, 
         False, False, False, False, True,
         False, False, False, False, False,
         False, False, False, False, False, 
        
         True, True, False, False, False, 
         True, False, False, True, False, 
         False, False, False, False, False, 
         False, True, True, False, True,

         False, True, False, False, False, 
         False, False, True, True, False, 
         False, True, True, False, False, 
         False, False, False, False, False, 

         False, False, True, False, False, 
         False, True, True, False, False, 
         False, False, False, True, False, 
         False, False, True, True, False]
print(labeled_docs['innovation_plan'].value_counts())
labeled_docs.head(5)

False    109
True      31
Name: innovation_plan, dtype: int64


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,level,link,text,title,type,innovation_plan
0,0,2683,2683,2683.0,Second,https://www.celinaisd.com/wp-documents/Require...,Date Run: Program: FIN1750 Cnty Dist: Page: 1 ...,Celina ISD,pdf,False
1,1,1910,1910,1910.0,Second,https://www.celinaisd.com/wp-documents/Require...,BUDGET RECEIVED TO DATE REMAINING PERCENT REMA...,Celina ISD,pdf,False
2,2,455,455,455.0,Second,https://1.cdn.edl.io/BoIQrMqDBRiGKjWEV3Sjf2tgY...,No text,West ISD,pdf,False
3,3,271,271,271.0,First,http://images.pcmac.org/Uploads/CalallenISD/Ca...,Calallen ISD: Put Us to the Test; We Are the B...,Calallen ISD,pdf,True
4,4,1020,1020,1020.0,Second,https://4.files.edl.io/436e/05/11/18/191230-4e...,Microsoft Word - Alto ISD Calendar 2018-2019 J...,Alto ISD,pdf,False


This adds extra trues to data used for training, but not testing. 

In [5]:
if add_extra_trues:
    docs_true = pd.read_csv(os.path.join(os.getcwd(), '..', '..', 'data', 'text_scraped.csv'))
    docs_true = docs_true[docs_true.level == 'First']
    docs_true = docs_true[docs_true.text != 'UNAVAILABLE']
    docs_true = docs_true[docs_true.text != 'No text']
    docs_true = docs_true[~docs_true.text.str.contains('Page Not Found')]
    
    #docs_true = docs_true[docs_true.level == 'First'][docs_true.text != 'UNAVAILABLE'][docs_true.text != 'No text']['Page Not Found' not in docs_true.text]
    docs_true = docs_true.sample(n = num_new_true)
    docs_true['innovation_plan'] = True
    # when we split into training and testing, all of the trues will be in training. 
    labeled_docs = docs_true.append(labeled_docs, ignore_index = True, sort = False) 
    print('New number of labelled documents = ', len(labeled_docs))
    labeled_docs.head()

New number of labelled documents =  260


In [6]:
print('The first n-30 of documents are used for training. Training dataset statistics:')
split = int(len(labeled_docs) - 30)
print(labeled_docs.head(split)['innovation_plan'].value_counts())

print('The last 30 of documents are used for testing. Testing dataset statistics:')
print(labeled_docs.tail(30)['innovation_plan'].value_counts())

The first n-30 of documents are used for training. Training dataset statistics:
True     143
False     87
Name: innovation_plan, dtype: int64
The last 30 of documents are used for testing. Testing dataset statistics:
False    22
True      8
Name: innovation_plan, dtype: int64


## Train Classifier and Save

In [8]:
texts, cats = classify.reformat_cat_from_df(df = labeled_docs,
                                      text_col = 'text',
                                      label_col = 'innovation_plan',
                                      category = 'INNOVATION')

texts = [i[:10000] for i in texts]

In [9]:
output_dir = os.path.join(os.getcwd(), '..', '..', 'data', 'document_classifier')

In [10]:
test_model = classify.train_classifier_and_evaluate(texts=texts, 
                                                    cats=cats, 
                                                    model=None, 
                                                    output_dir=output_dir, 
                                                    n_iter=4, 
                                                    categories=['INNOVATION'])

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


Created blank 'en' model
Loading IMDB data...
Using 260 examples (230 training, 30 evaluation)
Training the model...
LOSS 	  P  	  R  	  F  
Page not found – Celina ISD Skip to content Our Schools Home News Calendar District Our Schools About Athletics Community
{'INNOVATION': False}
INNOVATION 0.6227318048477173 False
District of Innovation Plan (FINAL) Inspiring & Empowering Learners INTRODUCTION HB 1842 was passed during the 84th Texas
{'INNOVATION': True}
INNOVATION 0.9999545812606812 True
SUNRAY ISD District of Innovation Plan 2 District of Innovation Plan Table of Contents Introduction……………
{'INNOVATION': True}
INNOVATION 0.9999545812606812 True
BUDGET RECEIVED TO DATE REMAINING PERCENT REMAINING REVENUES: 5751 REVENUE FROM MEALS SERVED 704,188.00$ 496,615.57$ 207,572.43$
{'INNOVATION': False}
INNOVATION 0.000957601354457438 False
Date Run: Program: FIN1750 Cnty Dist: Page: 1 of File ID: CFrom 01-01
{'INNOVATION': False}
INNOVATION 4.539787187241018e-05 False
Scanned from a Xerox

Page not found – Celina ISD Skip to content Our Schools Home News Calendar District Our Schools About Athletics Community
{'INNOVATION': False}
INNOVATION 0.0001987871655728668 False
District of Innovation Plan (FINAL) Inspiring & Empowering Learners INTRODUCTION HB 1842 was passed during the 84th Texas
{'INNOVATION': True}
INNOVATION 0.9999545812606812 True
SUNRAY ISD District of Innovation Plan 2 District of Innovation Plan Table of Contents Introduction……………
{'INNOVATION': True}
INNOVATION 0.9999545812606812 True
BUDGET RECEIVED TO DATE REMAINING PERCENT REMAINING REVENUES: 5751 REVENUE FROM MEALS SERVED 704,188.00$ 496,615.57$ 207,572.43$
{'INNOVATION': False}
INNOVATION 9.943953773472458e-05 False
Date Run: Program: FIN1750 Cnty Dist: Page: 1 of File ID: CFrom 01-01
{'INNOVATION': False}
INNOVATION 4.539787187241018e-05 False
Scanned from a Xerox multifunction device (5).pdf
{'INNOVATION': False}
INNOVATION 0.023033620789647102 False
No text
{'INNOVATION': False}
INNOVATION 4.5397

This movie sucked {'INNOVATION': 4.539787187241018e-05}


In [11]:
labeled_docs[labeled_docs['text'].str.contains('Graford')].text.values[0]

'Graford Independent School District District of Innovation Plan Composed February 2017 District of Innovation Committee Wes Corzine – Elementary Principal Lori Henderson – Secondary Principal Ty Tabor – Athletic Director Jamye Hayes – Elementary Teacher Chris Thompson – Director of Technology District of Innovation Timeline December 15 Board Resolution to start process January 26 Public Hearing Board Appoints district committee February 2, 6, 9, 17 Committee meets February 14, 16 Public Information meetings February 23 Committee reports to Board on recommendations March 9 DOI Plan approved by District SBDM and post to web-site March 30 Review of DOI plan by Board and plan to notify TEA of intent April 27 Board final approval of DOI Paperwork submitted to TEA District of Innovation – HB 1842 The District of Innovation concept, passed by the 84th Legislative Session in House Bill 1842, gives traditional independent school districts most of the flexibilities available to Texas open-enrol

In [12]:
import spacy
nlp = spacy.load('/Users/kylieleblancKylie/dofis/code/exemptions/../../data/document_classifier')

In [13]:
nlp.get_pipe('textcat')

In [14]:
test_model

In [15]:
doc = nlp.tokenizer(labeled_docs[labeled_docs['text'].str.contains('MEXIA')].text.values[0])

In [16]:
for i, doc in enumerate(test_model.pipe([doc])):
    print(doc[:20])
    for label, score in doc.cats.items():
        print(label, score)

District Of Innovation Plan MEXIA ISD District Of Innovation Plan 2017-2022 Board Approved: 3/27/2017 INTRODUCTION House Bill
INNOVATION 0.9999545812606812


In [17]:
labeled_docs[labeled_docs['text'].str.contains('Graford')].text.values[0]

'Graford Independent School District District of Innovation Plan Composed February 2017 District of Innovation Committee Wes Corzine – Elementary Principal Lori Henderson – Secondary Principal Ty Tabor – Athletic Director Jamye Hayes – Elementary Teacher Chris Thompson – Director of Technology District of Innovation Timeline December 15 Board Resolution to start process January 26 Public Hearing Board Appoints district committee February 2, 6, 9, 17 Committee meets February 14, 16 Public Information meetings February 23 Committee reports to Board on recommendations March 9 DOI Plan approved by District SBDM and post to web-site March 30 Review of DOI plan by Board and plan to notify TEA of intent April 27 Board final approval of DOI Paperwork submitted to TEA District of Innovation – HB 1842 The District of Innovation concept, passed by the 84th Legislative Session in House Bill 1842, gives traditional independent school districts most of the flexibilities available to Texas open-enrol

In [18]:
nlp(labeled_docs[labeled_docs['text'].str.contains('Graford')].text.values[0]).cats

{'INNOVATION': 0.9999545812606812}

In [19]:
[i.cats for i in map(nlp, labeled_docs.head()['text'])]

[{'INNOVATION': 0.9999545812606812},
 {'INNOVATION': 0.9999545812606812},
 {'INNOVATION': 0.9999545812606812},
 {'INNOVATION': 0.9999545812606812},
 {'INNOVATION': 0.9999545812606812}]

In [15]:
labeled_docs.head()['text']

0    Final-- District of Innovation—Approved(unanim...
1    Cleburne Independent School District Local Inn...
2    Chico Independent School District P.O. Box 95 ...
3    Douglass ISD District of Innovation Plan (HB 1...
4    (Expires March 27, 2022) (Expires March 27, 20...
Name: text, dtype: object

In [43]:
docs_true.text[8]

"Cherokee Independent School District District of Innovation Plan Introduction In order to take advantage of the opportunities afforded to Independent School Districts by HB 1842, Cherokee ISD is seeking to modify Education Code requirements at the local level in order to better meet the needs of our student population. Cherokee ISD is eligible for designation as an innovation district because the district's most recent performance rating under the Texas Education Code (TEC), §39.054, is at least Academically Acceptable, as indicated in the applicable year's academic accountability manual adopted under TEC §97.1001. Cherokee ISD is seeking to become a District of Innovation in order to facilitate areas of concern created by traditional public school restrictions imposed by the legislature through various laws enacted. Those areas specifically addressed in this plan are Teacher Certification requirements, Probationary Contracts, Teacher and Principal Appraisals and the First Day of Inst